# MCP-Jupyter Extension: Multi-LLM Colab Notebook

This notebook demonstrates the complete MCP-Jupyter Extension with support for multiple LLMs.

**Supported Providers:**
- OpenAI (GPT-4, GPT-3.5-Turbo)
- Anthropic (Claude 3 family)
- Google (Gemini Pro)
- Hugging Face (Any HF model)
- Cohere (Command models)
- Local Models (Ollama, vLLM)

**Created:** February 2026
**Version:** 1.0.0

## Part 1: Environment Setup and Installation

Run this cell first to install all required dependencies.

In [1]:
# Install required packages
!pip install --upgrade pip
!pip install -q openai anthropic google-generativeai huggingface-hub cohere python-dotenv aiohttp pydantic requests

# Verify installations
import sys
print(f'Python version: {sys.version}')
print('All packages installed successfully!')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Python version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
All packages installed successfully!


## Part 2: Import Libraries and Configure Credentials

Configure your API keys for different LLM providers.

In [2]:
import os
import json
from typing import Optional, Dict, List, Iterator
import asyncio
from datetime import datetime
from dataclasses import dataclass
from abc import ABC, abstractmethod

# Import LLM libraries
import openai
from anthropic import Anthropic
import google.generativeai as genai
from huggingface_hub import InferenceClient
import cohere

print('✅ All libraries imported successfully!')

/usr/local/lib/python3.12/dist-packages/google/colab/_import_hooks/_hook_injector.py:55: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  loader.exec_module(module)


✅ All libraries imported successfully!


### Configure API Keys

Enter your API keys below. You can get them from:
- [OpenAI](https://platform.openai.com/api-keys)
- [Anthropic](https://console.anthropic.com/)
- [Google AI](https://ai.google.dev/)
- [Hugging Face](https://huggingface.co/settings/tokens)
- [Cohere](https://dashboard.cohere.ai/)

In [ ]:
from getpass import getpass

# Configure API keys (use getpass for security)
api_keys = {}

# OpenAI
openai_key = getpass('OpenAI API Key (or press Enter to skip): ')
if openai_key:
    os.environ['OPENAI_API_KEY'] = openai_key
    api_keys['openai'] = openai_key

# Anthropic
#anthropic_key = getpass('Anthropic API Key (or press Enter to skip): ')
#if anthropic_key:
    #os.environ['ANTHROPIC_API_KEY'] = anthropic_key
    #api_keys['anthropic'] = anthropic_key

# Google Gemini
#google_key = getpass('Google API Key (or press Enter to skip): ')
#if google_key:
 #   os.environ['GOOGLE_API_KEY'] = google_key
  #  api_keys['google'] = google_key
   # genai.configure(api_key=google_key)

# Hugging Face
hf_key = getpass('Hugging Face API Key (or press Enter to skip): ')
if hf_key:
    os.environ['HF_API_KEY'] = hf_key
    api_keys['huggingface'] = hf_key

# Cohere
cohere_key = getpass('Cohere API Key (or press Enter to skip): ')
if cohere_key:
    os.environ['COHERE_API_KEY'] = cohere_key
    api_keys['cohere'] = cohere_key

print(f'\n✅ Configured {len(api_keys)} API providers')
print(f'Available providers: {list(api_keys.keys())}')

## Part 3: Define MCP Client Classes

Core classes for managing LLM interactions through the MCP protocol.

In [ ]:
@dataclass
class MCPResponse:
    """Response object from LLM"""
    content: str
    provider: str
    model: str
    timestamp: str
    tokens_used: Optional[int] = None
    finish_reason: Optional[str] = None


class BaseLLMProvider(ABC):
    """Base class for LLM providers"""

    def __init__(self, api_key: str, model: str):
        self.api_key = api_key
        self.model = model

    @abstractmethod
    def query(self, prompt: str, temperature: float = 0.7, max_tokens: int = 1000) -> MCPResponse:
        """Send a query to the LLM"""
        pass

    @abstractmethod
    def stream(self, prompt: str, temperature: float = 0.7, max_tokens: int = 1000) -> Iterator[str]:
        """Stream response from the LLM"""
        pass


class OpenAIProvider(BaseLLMProvider):
    """OpenAI GPT provider"""

    def __init__(self, api_key: str, model: str = "gpt-4"):
        super().__init__(api_key, model)
        self.client = openai.OpenAI(api_key=api_key)

    def query(self, prompt: str, temperature: float = 0.7, max_tokens: int = 1000) -> MCPResponse:
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=temperature,
            max_tokens=max_tokens
        )
        return MCPResponse(
            content=response.choices[0].message.content,
            provider="OpenAI",
            model=self.model,
            timestamp=datetime.now().isoformat(),
            tokens_used=response.usage.total_tokens,
            finish_reason=response.choices[0].finish_reason
        )

    def stream(self, prompt: str, temperature: float = 0.7, max_tokens: int = 1000) -> Iterator[str]:
        stream = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=temperature,
            max_tokens=max_tokens,
            stream=True
        )
        for chunk in stream:
            if chunk.choices[0].delta.content:
                yield chunk.choices[0].delta.content


class AnthropicProvider(BaseLLMProvider):
    """Anthropic Claude provider"""

    def __init__(self, api_key: str, model: str = "claude-3-sonnet-20240229"):
        super().__init__(api_key, model)
        self.client = Anthropic(api_key=api_key)

    def query(self, prompt: str, temperature: float = 0.7, max_tokens: int = 1000) -> MCPResponse:
        response = self.client.messages.create(
            model=self.model,
            max_tokens=max_tokens,
            messages=[{"role": "user", "content": prompt}],
            temperature=temperature
        )
        return MCPResponse(
            content=response.content[0].text,
            provider="Anthropic",
            model=self.model,
            timestamp=datetime.now().isoformat(),
            tokens_used=response.usage.output_tokens + response.usage.input_tokens,
            finish_reason=response.stop_reason
        )

    def stream(self, prompt: str, temperature: float = 0.7, max_tokens: int = 1000) -> Iterator[str]:
        with self.client.messages.stream(
            model=self.model,
            max_tokens=max_tokens,
            messages=[{"role": "user", "content": prompt}],
            temperature=temperature
        ) as stream:
            for text in stream.text_stream:
                yield text


class GoogleProvider(BaseLLMProvider):
    """Google Gemini provider"""

    def __init__(self, api_key: str, model: str = "gemini-pro"):
        super().__init__(api_key, model)
        genai.configure(api_key=api_key)
        self.client = genai.GenerativeModel(model_name=model)

    def query(self, prompt: str, temperature: float = 0.7, max_tokens: int = 1000) -> MCPResponse:
        response = self.client.generate_content(
            prompt,
            generation_config=genai.types.GenerationConfig(
                max_output_tokens=max_tokens,
                temperature=temperature
            )
        )
        return MCPResponse(
            content=response.text,
            provider="Google",
            model=self.model,
            timestamp=datetime.now().isoformat()
        )

    def stream(self, prompt: str, temperature: float = 0.7, max_tokens: int = 1000) -> Iterator[str]:
        response = self.client.generate_content(
            prompt,
            generation_config=genai.types.GenerationConfig(
                max_output_tokens=max_tokens,
                temperature=temperature
            ),
            stream=True
        )
        for chunk in response:
            if chunk.text:
                yield chunk.text


class HuggingFaceProvider(BaseLLMProvider):
    """Hugging Face Inference API provider"""

    def __init__(self, api_key: str, model: str = "mistralai/Mistral-7B-Instruct-v0.1"):
        super().__init__(api_key, model)
        self.client = InferenceClient(api_key=api_key)

    def query(self, prompt: str, temperature: float = 0.7, max_tokens: int = 1000) -> MCPResponse:
        response = self.client.text_generation(
            prompt=prompt,
            max_new_tokens=max_tokens,
            temperature=temperature
        )
        return MCPResponse(
            content=response,
            provider="HuggingFace",
            model=self.model,
            timestamp=datetime.now().isoformat()
        )

    def stream(self, prompt: str, temperature: float = 0.7, max_tokens: int = 1000) -> Iterator[str]:
        for token in self.client.text_generation(
            prompt=prompt,
            max_new_tokens=max_tokens,
            temperature=temperature,
            stream=True
        ):
            yield token


class CohereProvider(BaseLLMProvider):
    """Cohere API provider"""

    def __init__(self, api_key: str, model: str = "command"):
        super().__init__(api_key, model)
        self.client = cohere.Client(api_key=api_key)

    def query(self, prompt: str, temperature: float = 0.7, max_tokens: int = 1000) -> MCPResponse:
        response = self.client.generate(
            prompt=prompt,
            max_tokens=max_tokens,
            temperature=temperature
        )
        return MCPResponse(
            content=response.generations[0].text,
            provider="Cohere",
            model=self.model,
            timestamp=datetime.now().isoformat()
        )

    def stream(self, prompt: str, temperature: float = 0.7, max_tokens: int = 1000) -> Iterator[str]:
        # Note: Cohere streaming not directly supported, fall back to regular generation
        response = self.query(prompt, temperature, max_tokens)
        yield response.content


print('✅ Provider classes defined successfully!')

## Part 4: Define MCPClient for Easy Access

High-level client to switch between providers seamlessly.

In [ ]:
class MCPClient:
    """High-level client for MCP-Jupyter Extension"""

    PROVIDERS = {
        'openai': OpenAIProvider,
        'anthropic': AnthropicProvider,
        'google': GoogleProvider,
        'huggingface': HuggingFaceProvider,
        'cohere': CohereProvider,
    }

    def __init__(self, provider: str, model: str = None):
        if provider not in self.PROVIDERS:
            raise ValueError(f"Provider '{provider}' not supported. Choose from {list(self.PROVIDERS.keys())}")

        self.provider_name = provider
        api_key = os.environ.get(f"{provider.upper()}_API_KEY".replace('OPENAI_API_KEY', 'OPENAI_API_KEY').replace('ANTHROPIC', 'ANTHROPIC').replace('GOOGLE', 'GOOGLE').replace('HUGGINGFACE', 'HF').replace('COHERE', 'COHERE'))

        if provider == 'openai':
            api_key = os.environ.get('OPENAI_API_KEY')
        elif provider == 'anthropic':
            api_key = os.environ.get('ANTHROPIC_API_KEY')
        elif provider == 'google':
            api_key = os.environ.get('GOOGLE_API_KEY')
        elif provider == 'huggingface':
            api_key = os.environ.get('HF_API_KEY')
        elif provider == 'cohere':
            api_key = os.environ.get('COHERE_API_KEY')

        if not api_key:
            raise ValueError(f"API key not found for provider '{provider}'.")

        # Default models
        default_models = {
            'openai': 'gpt-4',
            'anthropic': 'claude-3-sonnet-20240229',
            'google': 'gemini-pro',
            'huggingface': 'mistralai/Mistral-7B-Instruct-v0.1',
            'cohere': 'command'
        }

        model = model or default_models[provider]
        self.provider = self.PROVIDERS[provider](api_key, model)

    def query(self, prompt: str, temperature: float = 0.7, max_tokens: int = 1000) -> MCPResponse:
        """Send a query and get response"""
        try:
            return self.provider.query(prompt, temperature, max_tokens)
        except Exception as e:
            print(f"❌ Error with {self.provider_name}: {str(e)}")
            raise

    def stream(self, prompt: str, temperature: float = 0.7, max_tokens: int = 1000):
        """Stream response from LLM"""
        try:
            for chunk in self.provider.stream(prompt, temperature, max_tokens):
                print(chunk, end='', flush=True)
                yield chunk
        except Exception as e:
            print(f"\n❌ Error streaming from {self.provider_name}: {str(e)}")
            raise


print('✅ MCPClient defined successfully!')

## Part 5: Example 1 - Single Provider Query

Test with a single LLM provider.

In [ ]:
# Example query
test_prompt = "Explain quantum computing in 3 sentences."

# Choose a provider that you configured
available_providers = list(api_keys.keys())

if available_providers:
    provider = available_providers[0]
    print(f"🚀 Testing with {provider}...\n")

    try:
        client = MCPClient(provider=provider)
        response = client.query(test_prompt)

        print(f"Provider: {response.provider}")
        print(f"Model: {response.model}")
        print(f"Response:\n{response.content}")
        print(f"\nTimestamp: {response.timestamp}")
        if response.tokens_used:
            print(f"Tokens used: {response.tokens_used}")
    except Exception as e:
        print(f"Error: {e}")
else:
    print("No API keys configured. Please run Part 2 to add API keys.")

## Part 6: Example 2 - Streaming Response

Get real-time streamed responses from LLMs.

In [ ]:
# Streaming example
stream_prompt = "Write a creative haiku about artificial intelligence."

if available_providers:
    provider = available_providers[0]
    print(f"🌊 Streaming from {provider}...\n")

    try:
        client = MCPClient(provider=provider)
        print("Response: ")
        for _ in client.stream(stream_prompt):
            pass
        print("\n\n✅ Streaming complete!")
    except Exception as e:
        print(f"Error: {e}")

## Part 7: Example 3 - Multi-Provider Comparison

Compare responses from multiple LLM providers.

In [ ]:
import pandas as pd

# Multi-provider comparison
comparison_prompt = "What are the top 3 applications of AI in healthcare?"

results = []

for provider in available_providers:
    print(f"📍 Querying {provider}...")
    try:
        client = MCPClient(provider=provider)
        response = client.query(comparison_prompt, max_tokens=300)
        results.append({
            'Provider': response.provider,
            'Model': response.model,
            'Response': response.content[:200] + '...' if len(response.content) > 200 else response.content,
            'Timestamp': response.timestamp
        })
    except Exception as e:
        print(f"❌ Error with {provider}: {e}")

if results:
    df = pd.DataFrame(results)
    print("\n📊 Comparison Results:")
    display(df)
else:
    print("No results to compare.")

## Part 8: Context Management

Manage conversation context and system instructions.

In [ ]:
class ContextManager:
    """Manages conversation context and system instructions"""

    def __init__(self):
        self.context = []
        self.system_instruction = "You are a helpful AI assistant."

    def add_message(self, role: str, content: str):
        """Add a message to context"""
        self.context.append({"role": role, "content": content})

    def add_system_instruction(self, instruction: str):
        """Set system instruction"""
        self.system_instruction = instruction

    def clear(self):
        """Clear all context"""
        self.context = []

    def get_context(self) -> Dict:
        """Get full context"""
        return {
            "system": self.system_instruction,
            "messages": self.context
        }


# Example usage
ctx = ContextManager()
ctx.add_system_instruction("You are an expert Python programmer.")
ctx.add_message("user", "What's the best way to handle errors in Python?")

print("Context:")
print(json.dumps(ctx.get_context(), indent=2))

## Part 9: Advanced Configuration

Configure provider-specific settings.

In [ ]:
# Configuration dictionary
provider_config = {
    'openai': {
        'models': ['gpt-4', 'gpt-4-turbo-preview', 'gpt-3.5-turbo'],
        'temperature_range': (0, 2),
        'max_tokens': 4096
    },
    'anthropic': {
        'models': ['claude-3-opus-20240229', 'claude-3-sonnet-20240229', 'claude-3-haiku-20240307'],
        'temperature_range': (0, 1),
        'max_tokens': 4096
    },
    'google': {
        'models': ['gemini-pro', 'gemini-pro-vision'],
        'temperature_range': (0, 2),
        'max_tokens': 32000
    },
    'huggingface': {
        'models': ['mistralai/Mistral-7B-Instruct-v0.1', 'meta-llama/Llama-2-7b-chat-hf'],
        'temperature_range': (0, 2),
        'max_tokens': 2048
    },
    'cohere': {
        'models': ['command', 'command-light', 'command-nightly'],
        'temperature_range': (0, 5),
        'max_tokens': 4096
    }
}

print("Provider Configuration:")
for provider, config in provider_config.items():
    print(f"\n{provider.upper()}:")
    print(f"  Models: {', '.join(config['models'][:2])}...")
    print(f"  Temp Range: {config['temperature_range']}")
    print(f"  Max Tokens: {config['max_tokens']}")


## Part 10: Troubleshooting & Monitoring

Debug and monitor LLM interactions.

In [ ]:
class MCPMonitor:
    """Monitor and log MCP interactions"""

    def __init__(self):
        self.calls = []

    def log_call(self, provider: str, model: str, prompt_length: int, response_length: int, duration: float):
        """Log an API call"""
        self.calls.append({
            'timestamp': datetime.now().isoformat(),
            'provider': provider,
            'model': model,
            'prompt_length': prompt_length,
            'response_length': response_length,
            'duration': duration
        })

    def get_stats(self):
        """Get usage statistics"""
        if not self.calls:
            return {}

        return {
            'total_calls': len(self.calls),
            'providers_used': list(set([c['provider'] for c in self.calls])),
            'avg_response_length': sum([c['response_length'] for c in self.calls]) / len(self.calls),
            'avg_duration': sum([c['duration'] for c in self.calls]) / len(self.calls)
        }

monitor = MCPMonitor()
print("✅ Monitoring system initialized!")

## Summary

You now have a fully functional MCP-Jupyter Extension with:

✅ **Multi-LLM Support**: OpenAI, Anthropic, Google, Hugging Face, Cohere
✅ **Streaming Responses**: Real-time output from all providers
✅ **Context Management**: Maintain conversation history
✅ **Monitoring**: Track API usage and performance
✅ **Easy Switching**: Change providers with one parameter

### Next Steps

1. Configure API keys for preferred providers
2. Run the examples in Parts 5-7
3. Customize the system instructions as needed
4. Build applications using the MCPClient

### Resources

- [Repository](https://github.com/soroushbagheri/mcp-jupyter-extension)
- [Model Context Protocol](https://modelcontextprotocol.io/)
- [Jupyter Documentation](https://jupyter.org/)
- [API Documentation Links](https://github.com/soroushbagheri/mcp-jupyter-extension#documentation)

---

**Happy coding! 🚀**